In [21]:
import numpy as np
from sympy import Matrix, zoo

class SimplexSolver:
    def __init__(self, filename):
        self.A, self.b, self.c, self.z = self._read_csv(filename)
        self.b_org = self.b.copy()
        self.epsilon = 0.5  # Equivalent to pow(2, -1)
        self.z = self.findFeasiblePoint()


    def _read_csv(self, filename):
        '''
        Reads the input matrix A, vectors b and c, and initial point z from a CSV file.
        Input:
            - CSV file with m+1 rows and n+1 columns.
            - The first row (excluding the last element) is the cost vector c of length n.
            - The last column (excluding the top element) is the constraint vector b of length m.
            - Rows 2 to m+1 and columns 1 to n form the matrix A of size m*n.
        '''
        try:
            with open(filename, 'r') as input_file:
                # Read the entire file into a numpy array
                data = np.loadtxt(input_file, delimiter=',', dtype=float)

            # Extract vector c (first row excluding the last element)
            c = data[0, :-1]

            # Extract vector b (last column excluding the top element)
            b = data[1:, -1]

            # Extract matrix A (rows 2 to m+1 and columns 1 to n)
            A = data[1:, :-1]

            # Initial feasible point z is set to None by default
            z = None  # Modify this as required based on the problem context

            return A, b, c, z
        except FileNotFoundError:
            raise FileNotFoundError(f'File "{filename}" not found.')
        except Exception as e:
            raise ValueError(f'Error reading file "{filename}": {e}')

    def _test_feasibility(self, z, A=None, b=None):
        '''
        Checks whether a point z is a feasible solution for the system or not.
        '''
        if A is None:
            A = self.A
        if b is None:
            b = self.b
        return np.all(np.matmul(A, z) <= b)


    def findFeasiblePoint(self):
        '''
        Finds a point z that is inside the feasible region of the given system.
        '''
        m, n = self.A.shape
        # print("A", self.A)

        # Check if the origin is a feasible point
        if all([x >= 0 for x in self.b]):
            return np.zeros(n, dtype=float)
        
        # Formulate a new optimization problem
        P = np.empty((m + 1, n + 1), dtype=float)    
        for x in self.A:
            x = np.append(x, 1)
            P = np.vstack((P, x))
        zeros = np.zeros(n + 1)
        zeros[n] = -1
        P = np.vstack((P, zeros))        
        P = P[m + 1:]
        temp = self.b
        q = np.append(temp, -1 * np.min(self.b))
        z = np.zeros(n, dtype=float)
        z = np.append(z, np.min(self.b))
        r = np.zeros(n, dtype=float)
        r = np.append(r, 1)

        # print("Z,P,q", z, P, q)
        # Step 1a: Check for feasibility of the Initial Point
        if not self._test_feasibility(z=z, A=P, b=q):
            # print("RETURN NOPONONONO")
            return []

        solved = False
        epsilon = pow(2, -5)
        q_org = q

        while not solved:

            # Step 1b: Move towards the first vertex
            while not self._test_vertex(z=z, A=P, b=q):
                z = self._move(z=z, A=P, b=q)
                print("moving towards", z)

            # Test for degeneracy of the vertex
            if self._test_degeneracy(z=z, A=P, b=q):
                q = self._remove_degeneracy(q_org.copy(), epsilon)
                epsilon *= 2
                continue

            # Step 1c: March through every vertex and check for optimum along with degeneracy
            solution = True
            degenerate = False
            while solution and (not self._test_optimum(A=P, b=q, z=z, c=r)):
                z, solution = self._optimize(A=P, b=q, z=z, c=r)

                # If the new polytope is unbounded, reverse the direction and find the feasible point
                if not solution:
                    P1, _, P2, q2 = self._split_rows(P, q, z)
                    u = None
                    P1_inverse = np.linalg.inv(P1)
                    P1_inverse = np.transpose(P1_inverse)
                    for x in P1_inverse:
                        if np.dot((-1) * x, r.T) > 0:
                            u = x.T
                            break
                    temp = np.dot(P2, u)
                    for i in range(len(temp)):
                        if temp[i] == 0:
                            temp[i] = 1e-16
                    alpha = (q2 - np.dot(P2, z)) / temp
                    alpha = min([a for a in alpha if a >= 0])
                    z = z - alpha * u

                    # Check for feasibility of the point
                    if z[-1] < 0:
                        z = []
                    else:
                        z = z[:-1]
                    break
                
                elif self._test_degeneracy(z=z, A=P, b=q):
                    q = self._remove_degeneracy(q_org.copy(), epsilon)
                    degenerate = True
                    epsilon *= 2
                    break
                
                else:
                    continue

            if not degenerate:
                solved = True

        # Convert the solution back to the original solution, if it exists
        if solution:
            X, y = [], []
            for i in range(len(P)):
                if np.round(np.dot(P[i], z), 3) == np.round(q[i], 3):
                    X.append(P[i])
                    y.append(q_org[i])
            z = np.dot(np.linalg.inv(X), y)

            # Check for feasibility of the point
            if z[-1] < 0:
                z = []
            else:
                z = z[:-1]
        
        return z

    def _split_rows(self, z, A=None, b=None):
        '''
        Splits the matrices A and b into sets of tight and untight rows with respect to a given point z.
        '''
        # print("in split rows")
        if A is None:
            A = self.A
        if b is None:
            b = self.b
        
        A1, A2, b1, b2 = [], [], [], []
        Az = np.dot(A, z)
        for i in range(len(Az)):
            if np.isclose(np.dot(A[i], z), b[i]):
                A1.append(A[i])
                b1.append(b[i])
            else:
                A2.append(A[i])
                b2.append(b[i])
        return np.array(A1), np.array(b1), np.array(A2), np.array(b2)

    def _test_vertex(self, z, A=None, b=None):
        '''
        Checks whether a point z is a vertex of the feasible region or not.
        '''
        # print("in test vertex")
        if A is None:
            A = self.A
        if b is None:
            b = self.b

        A1, _, _, _ = self._split_rows(z=z, A=A, b=b)

        # print("A1", A1)
        rank_A = np.linalg.matrix_rank(A)
        num_tight_rows = A1.shape[0]
        return rank_A <= num_tight_rows



    def _move(self, z, A=None, b=None):
        '''
        Moves the feasible point towards a vertex of the feasible region by one step size.
        '''
        if A is None:
            A = self.A
        if b is None:
            b = self.b

        try:
            m, n = A.shape
            A1, _, A2, b2 = self._split_rows(z=z, A=A, b=b)
            A1 = A1.reshape((A1.shape[0], n))

            # Compute the direction to move the feasible point
            nullspace = np.array(Matrix(A1).nullspace())
            if nullspace.size == 0:
                raise ValueError('Nullspace of A1 is empty; cannot move towards vertex.')
            scale = np.random.randint(1, 4, nullspace.shape[0]).reshape(nullspace.shape[0], 1, 1)
            vector = np.array(np.sum(nullspace * scale, axis=0)).astype(float)
            u = vector / np.linalg.norm(vector)

            # Compute the step size 'alpha'
            temp = np.dot(A2, u)
            temp[np.isclose(temp, 0)] = 1e-32  # Avoid division by zero
            alpha_values = (b2 - np.dot(A2, z)) / temp.T
            alpha_values = alpha_values[0]
            alpha_values = np.array([a for a in alpha_values if not ((a > 1e6) or (-a) > 1e6)])
            if alpha_values.size == 0:
                raise ValueError('No valid alpha found; cannot move towards vertex.')
            alpha = alpha_values[np.argmin(np.abs(alpha_values.copy()))]

            # Update the feasible point by one step size
            u = u.T
            z_new = z + alpha * u
            z_new = z_new[0]

            return z_new
        except Exception as e:
            raise RuntimeError(f'Error in moving towards vertex: {e}')

    def _test_degeneracy(self, z, A=None, b=None):
        '''
        Checks whether a vertex z of the feasible region is degenerate.
        '''
        if A is None:
            A = self.A
        if b is None:
            b = self.b

        A1, _, _, _ = self._split_rows(z=z, A=A, b=b)
        return np.linalg.matrix_rank(A) < A1.shape[0]

    def _remove_degeneracy(self, b, epsilon=None):
        '''
        Removes degeneracy from the feasible region by adding powers of a small positive number to vector b.
        '''
        if epsilon is None:
            epsilon = self.epsilon 
            
        add = self.epsilon
        for i in range(len(b)):
            b[i] += add
            add *= self.epsilon
        return b

    def _test_optimum(self, z, A=None, b=None, c=None):
        '''
        Checks whether a vertex z is an optimum vertex or not.
        '''

        if A is None:
            A = self.A
        if b is None:
            b = self.b
        if c is None:
            c = self.c

        try:
            A1, _, _, _ = self._split_rows(z=z, A=A, b=b)
            beta = np.dot(np.linalg.inv(A1.T), c.T)
            return np.all(beta >= 0)
        except np.linalg.LinAlgError:
            raise np.linalg.LinAlgError('Matrix A1.T is singular; cannot compute beta.')
        except Exception as e:
            raise RuntimeError(f'Error in testing for optimum: {e}')

    def _optimize(self, z, A=None, b=None, c=None):
        '''
        Optimizes the cost function by moving to the next optimum vertex.
        '''
        if A is None:
            A = self.A
        if b is None:
            b = self.b
        if c is None:
            c = self.c

        try:
            A1, _, A2, b2 = self._split_rows(z=z, A=A, b=b)

            # Compute the direction to move in order to reach a more optimum vertex
            A1_inverse = np.linalg.inv(A1)
            A1_inverse = np.transpose(A1_inverse)
            u = None
            for x in A1_inverse:
                if np.dot(-x, c.T) > 0:
                    u = x.T
                    break
            if u is None:
                raise ValueError('No direction found to optimize the cost function.')

            # Compute the step size 'alpha'
            temp = np.dot(A2, u)
            temp[np.isclose(temp, 0)] = 1e-16  # Avoid division by zero
            alpha_values = (np.dot(A2, z) - b2) / temp
            alpha_values = [x for x in alpha_values if x != zoo]
            if all([a < 0 for a in alpha_values]):
                return z, False
            alpha = min([x for x in alpha_values if x >= 0])

            # Update the vertex
            z_new = z - alpha * u

            return z_new, True
        except np.linalg.LinAlgError:
            raise np.linalg.LinAlgError('Matrix A1 is singular; cannot compute inverse.')
        except Exception as e:
            raise RuntimeError(f'Error in optimization step: {e}')

    def solve(self):
        '''
        Driver function which calls the subroutines according to the Simplex Algorithm.
        '''
        try:
            # Step 1: Check for feasibility of the Initial Point
            if not self._test_feasibility(self.z):
                raise ValueError(f'Initial point {self.z} is not feasible.')

            solved = False

            while not solved:
                # Step 2: Move towards the first vertex
                while not self._test_vertex(self.z):
                    self.z = self._move(self.z)

                # Test for degeneracy of the vertex
                if self._test_degeneracy(self.z):
                    self.b = self._remove_degeneracy(self.b_org.copy())
                    self.epsilon *= 2
                    continue

                # Print the value of the Cost function at this first vertex
                print(f'Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}')

                # Step 3: March through every vertex and check for optimum along with degeneracy
                solution = True
                degenerate = False
                while solution and (not self._test_optimum(self.z)):
                    self.z, solution = self._optimize(self.z)
                    if not solution:
                        raise ValueError('No solution found during optimization.')
                    elif self._test_degeneracy(self.z):
                        self.b = self._remove_degeneracy(self.b_org.copy())
                        degenerate = True
                        self.epsilon *= 2
                        break
                    else:
                        print(f'Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}')

                if not degenerate:
                    solved = True

            # Convert the solution back to the original solution, if it exists
            if solution:
                X, y = [], []
                for i in range(len(self.A)):
                    if np.isclose(np.dot(self.A[i], self.z), self.b[i]):
                        X.append(self.A[i])
                        y.append(self.b_org[i])
                X = np.array(X)
                y = np.array(y)
                try:
                    z_solution = np.linalg.solve(X, y)
                except np.linalg.LinAlgError:
                    raise np.linalg.LinAlgError('Final system is singular; cannot find unique solution.')
                print(f'Original Solution: Vertex {z_solution} --> Cost: {np.dot(z_solution, self.c)}')
        except Exception as e:
            print(f'Error: {e}')


In [22]:
if __name__ == '__main__':
    # Replace the filename with the path to your input CSV file
    filename = r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\4.Dataset.csv'
    try:
        solver = SimplexSolver(filename)
        solver.solve()
    except Exception as e:
        print(f'An error occurred: {e}')


moving towards [-3.  3. -4.]
Vertex [1. 3.] --> Cost: 10.0
Error: No solution found during optimization.
